# Synthetic tables

In [2]:
import os
import pypdfium2 as pdfium

pdf_dir = "../../benchmark_truth/synthetic_tables/separate_files/final"
pdf_texts = []

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        file_path = os.path.join(pdf_dir, filename)
        doc = pdfium.PdfDocument(file_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.get_page(page_num)
            text += page.get_textpage().get_text_range()
        pdf_texts.append({'filepath': file_path, 'text': text})

# pdf_texts now contains the extracted text for each PDF file

In [3]:
aktiva_structure_hgb = {
    'Anlagevermögen': {
        'Immaterielle Vermögensgegenstände': [
            'Selbst geschaffene gewerbliche Schutzrechte und ähnliche Rechte und Werte',
            'Geschäfts- oder Firmenwert',
            'geleistete Anzahlungen',
            'entgeltlich erworbene Konzessionen, gewerbliche Schutzrechte und ähnliche Rechte und Werte sowie Lizenzen an solchen Rechten und Werten'
        ], 
        'Sachanlagen': [
            'Grundstücke, grundstücksgleiche Rechte und Bauten einschließlich der Bauten auf fremden Grundstücken',
            'Technische Anlagen und Maschinen',
            'Andere Anlagen, Betriebs- und Geschäftsausstattung',
            'geleistete Anzahlungen und Anlagen im Bau'
        ],
        'Finanzanlagen': [
            'Sonstige Finanzanlagen',
            'Anteile an verbundenen Unternehmen',
            'Ausleihungen an verbundene Unternehmen',
            'Beteiligungen',
            'Ausleihungen an Unternehmen, mit denen ein Beteiligungsverhältnis besteht',
            'Wertpapiere des Anlagevermögens',
            'Sonstige Ausleihungen'
        ]
    },
    'Umlaufvermögen': {
        'Vorräte': [
            'Roh-, Hilfs- und Betriebsstoffe',
            'Unfertige Erzeugnisse, unfertige Leistungen',
            'Fertige Erzeugnisse und Waren',
            'Geleistete Anzahlungen'
        ],
        'Forderungen und sonstige Vermögensgegenstände': [
            'Forderungen aus Lieferungen und Leistungen',
            'Forderungen gegen verbundene Unternehmen',
            'Forderungen gegen Unternehmen, mit denen ein Beteiligungsverhältnis besteht',
            'Sonstige Vermögensgegenstände'
        ],
        'Wertpapiere': [
            'Anteile an verbundenen Unternehmen',
            'Sonstige Wertpapiere'
        ],
        'Kassenbestand, Bundesbankguthaben, Guthaben bei Kreditinstituten und Schecks': []
    },
    'Rechnungsabgrenzungsposten': dict(),
    'Aktive latente Steuern': dict(),
    'Aktiver Unterschiedsbetrag aus der Vermögensverrechnung': dict()
}

In [ ]:
pdf_texts[0]['filepath'] # very strange parsed text

'../../benchmark_truth/synthetic_tables/separate_files/final/aktiva_table__3_columns__span_False__thin_False__year_as_text__unit_in_first_cell_True__Mio. €__enumeration_False__shuffle_True__text_around_True__max_length_50__sum_in_same_row_False__0.pdf'

In [23]:
entry = pdf_texts[1]
first_text = entry['text']
print(first_text)

Aktiva Geschäftsjahr Vorjahr
Tsd. € Tsd. € Tsd. €
A. Anlagevermögen
I. Immaterielle Vermögensgegenstände
1. Selbst geschaffene gewerbliche Schutzrechte und
ähnliche Rechte und Werte
9.586,21 5.874,80
2. Geschäfts- oder Firmenwert 7.686,89 841,58
3. entgeltlich erworbene Konzessionen, gewerbliche
Schutzrechte und ähnliche Rechte und Werte sowie
Lizenzen an solchen Rechten und Werten
3.091,58 5.868,83
20.364,67 12.585,21
II. Sachanlagen
1. Grundstücke, grundstücksgleiche Rechte und
Bauten einschließlich der Bauten auf fremden
Grundstücken
2.900,39 8.697,91
2. Technische Anlagen und Maschinen 1.349,51 2.423,71
3. geleistete Anzahlungen und Anlagen im Bau 1.556,63 5.688,00
5.806,53 16.809,62
III. Finanzanlagen
1. Sonstige Finanzanlagen 69,79 5.202,43
2. Ausleihungen an verbundene Unternehmen 1.058,53 2.578,81
3. Beteiligungen 3.889,04 3.169,03
4. Ausleihungen an Unternehmen, mit denen ein
Beteiligungsverhältnis besteht
1.992,18 1.193,12
7.009,54 12.143,39
33.180,74 41.538,22
B. Umlaufvermö

In [24]:
import re

def extract_numbers_by_key(text, key):
    # Find all numbers in the text
    # Match numbers with optional thousands separators and decimal part
    # Search for the key followed by numbers, and extract the substring starting from the key
    text = text.replace('\n', ' ').replace('\r', '')  # Replace newlines with spaces for better regex matching
    if key not in text:
        # print(f"Key '{key}' not found in text.")
        pass
    match = re.search(rf'({key}\s*((?:\d{{1,3}}(?:[\s.]\d{{3}})*[,]\d+[\s]*)+))', text)
    if match:
        numbers_str = match.group(1)
        numbers = re.findall(r'\d{1,3}(?:[.,]\d{3})*[.,]\d+', numbers_str)
    else:
        numbers = []
    # print(f"Extracted numbers for key '{key}': {numbers}")
    return [float(num.replace('.', '').replace(',', '.')) for num in numbers]  # Replace comma with dot for float conversion

def add_row(extracted_rows, key1, key2, item, numbers):
    if len(numbers) > 0:
        if len(numbers) > 1:
            extracted_rows.append({'E1': key1, 'E2': key2, 'E3': item, 'year': numbers[0], 'previous_year': numbers[1]})
            if len(numbers) > 2:
                # print(f"Warning: More than two numbers found for {item}. Only the first two will be used.")
                pass
        else:
            extracted_rows.append({'E1': key1, 'E2': key2, 'E3': item, 'year': numbers[0], 'previous_year': None})
    else:
        extracted_rows.append({'E1': key1, 'E2': key2, 'E3': item, 'year': None, 'previous_year': None})

def extract_numbers(text):
    extracted_rows = []

    for key1, value1 in aktiva_structure_hgb.items():
        for key2, value2 in value1.items():
            for item in value2:
                # print(f"{key1} > {key2} > {item}")
                numbers = extract_numbers_by_key(text, item)
                # print(f"Numbers found: {numbers}")
                add_row(extracted_rows, key1, key2, item, numbers)
                numbers = []
            if len(value2) == 0:
                # print(f"{key1} > {key2} > (no items)")
                numbers = extract_numbers_by_key(text, key2)
                # print(f"Numbers found: {numbers}")
                add_row(extracted_rows, key1, key2, None, numbers)
                numbers = []
        if len(value1) == 0:
            # print(f"{key1} > (no subcategories)")
            numbers = extract_numbers_by_key(text, key1)
            # print(f"Numbers found: {numbers}")
            add_row(extracted_rows, key1, None, None, numbers)
            numbers = []

    return extracted_rows

In [25]:
import pandas as pd

extracted_rows = extract_numbers(first_text)
df_result = pd.DataFrame(extracted_rows)
df_result


,E1,E2,E3,year,previous_year
0,Anlagevermögen,Immaterielle Vermögensgegenstände,Selbst geschaffene gewerbliche Schutzrechte un...,9586.21,5874.80
1,Anlagevermögen,Immaterielle Vermögensgegenstände,Geschäfts- oder Firmenwert,7686.89,841.58
2,Anlagevermögen,Immaterielle Vermögensgegenstände,geleistete Anzahlungen,NaN,NaN
3,Anlagevermögen,Immaterielle Vermögensgegenstände,"entgeltlich erworbene Konzessionen, gewerblich...",3091.58,5868.83
4,Anlagevermögen,Sachanlagen,"Grundstücke, grundstücksgleiche Rechte und Bau...",2900.39,8697.91
5,Anlagevermögen,Sachanlagen,Technische Anlagen und Maschinen,1349.51,2423.71
6,Anlagevermögen,Sachanlagen,"Andere Anlagen, Betriebs- und Geschäftsausstat...",NaN,NaN
7,Anlagevermögen,Sachanlagen,geleistete Anzahlungen und Anlagen im Bau,1556.63,5688.00
8,Anlagevermögen,Finanzanlagen,Sonstige Finanzanlagen,69.79,5202.43
9,Anlagevermögen,Finanzanlagen,Anteile an verbundenen Unternehmen,NaN,NaN


In [96]:
import json
import nltk

ebnf_rows = pd.read_csv("../../benchmark_truth/real_tables/ebnf_rows.csv")

def parse_json(string):
    # Remove code block markers and extra whitespace, then parse as JSON
    json_str = string.strip()
    if json_str.startswith("```json"):
        json_str = json_str[len("```json"):].strip()
    if json_str.endswith("```"):
        json_str = json_str[:-3].strip()
    parsed_json = json.loads(json_str)
    return parsed_json


def replace_special_characters(text):
    # Replace German special characters in the input text
    return (
        text.replace("ä", "ae")
        .replace("ö", "oe")
        .replace("ü", "ue")
        .replace("Ä", "Ae")
        .replace("Ö", "Oe")
        .replace("Ü", "Ue")
        .replace("ß", "ss")
    )

def get_json_string(df, multiplier=1):
    df_rounded = df.copy()
    df_rounded = ebnf_rows.merge(df_rounded, how="left", on=["E1", "E2", "E3"])
    if len(df_rounded.columns) >= 2:
        last_two = df_rounded.columns[-2:]
        df_rounded = df_rounded.rename(columns={last_two[-2]: "year", last_two[-1]: "previous_year"})
    for col in ["year", "previous_year"]:
        if col in df_rounded.columns:
            df_rounded[col] = pd.to_numeric(df_rounded[col], errors='coerce')
            df_rounded[col] = df_rounded[col].apply(lambda x: (f"{x/multiplier:.2f}" if not isinstance(multiplier, dict) else (f"{(x/multiplier.get(col, 1)):.0f}" if multiplier.get(col, 1) != 1 else f"{x/multiplier.get(col, 1):.2f}")) if pd.notnull(x) else x)
    # print(df_rounded.head(5))
    # print(df_rounded.shape[0], "rows in the dataframe after merging with EBNF rows.")

    s = df_rounded.to_json(orient='records', indent=0, force_ascii=False)#.replace("null", '"null"')
    s_fixed = re.sub(r'("year":)"([0-9\.\-e]+)"', r'\1\2', s)
    s_fixed = re.sub(r'("previous_year":)"([0-9\.\-e]+)"', r'\1\2', s_fixed)
    json_str = replace_special_characters(s_fixed)

    prev_entry = ",{\"E1\":\"Anlagevermoegen\",\"E2\":\"Finanzanlagen\",\"E3\":\"Ausleihungen an Unternehmen, mit denen ein Beteiligungsverhaeltnis besteht\""
    beteilingungen_entry = ",{\"E1\":\"Anlagevermoegen\",\"E2\":\"Finanzanlagen\",\"E3\":\"Beteiligungen\""

    if beteilingungen_entry not in json_str:
        json_str = json_str.replace(prev_entry, beteilingungen_entry + ',"year":null,"previous_year":null}' + prev_entry)
        # print(f"Ground truth JSON string: \n{json_str}")
    
    return json_str

unit_list = {
    'EUR': 1, 
    '€': 1, 
    'Tsd. EUR': 1000, 
    'Mio. EUR': 1000000, 
    'TEUR': 1000, 
    'T€': 1000, 
    'Tsd. €': 1000, 
    'Mio. €': 1000000
}

def evaluate_single_result(df_result, entry):
    evaluation = {
        'json_error': False,
        'grammar_error': False,
    }

    entry_filepath = entry.get('filepath', 'unknown').replace('.pdf', '.csv')
    if '.csv' in entry_filepath:
        df = pd.read_csv(entry_filepath)
    elif '.xlsx' in entry_filepath:
        df = pd.read_excel(entry_filepath)
        
    df = df.map(lambda x: replace_special_characters(x) if isinstance(x, str) else x)

    if ("synth" in entry_filepath):
        unit = entry_filepath.split('__')[-7]
        multiplier = unit_list.get(unit, 1)
    else:
        T_in_year = entry.get('T_in_year', False)
        T_in_previous_year = entry.get('T_in_previous_year', False)
        multiplier = {'year': 1000 if T_in_year else 1, 'previous_year': 1000 if T_in_previous_year else 1}
        # multiplier = 1

    # rounds values matching with the one in the pdfs (synthetic tables important)
    # multiplier = 1  # Default multiplier, can be adjusted based on the unit in the entry
    json_str = get_json_string(df, multiplier=multiplier)

    try:
        truth = parse_json(json_str)
    except json.JSONDecodeError as e:
        raise ValueError(f"Error parsing JSON for truth: {json_str}\nError message: {e}")
    # pprint(truth)
    df_truth = pd.DataFrame(truth)
    # df_truth = df_truth.map(lambda x: x.lower() if isinstance(x, str) else x)
    # print(truth)

    df_result = df_result.copy()
    df_result = df_result.map(lambda x: replace_special_characters(x) if isinstance(x, str) else x)
    # print(df_result)

    # Join the ground truth and result dataframes on E1, E2, and E3 for comparison
    df_joined = pd.merge(
        df_truth,
        df_result,
        on=["E1", "E2", "E3"],
        how="left",
        suffixes=("_truth", "_result"),
        indicator=True
    )
    evaluation['df_joined'] = df_joined
    # print(df_joined[['E1', 'E2', 'E3', 'year_truth', 'year_result',]])

    evaluation['NA'] = {
        "true_positive": df_joined[(df_joined['_merge'] == 'both') & (df_joined['year_truth'].isna() & df_joined['year_result'].isna())].shape[0] + df_joined[(df_joined['_merge'] == 'both') & (df_joined['previous_year_truth'].isna() & df_joined['previous_year_result'].isna())].shape[0],
        "false_positive": df_joined[(df_joined['_merge'] == 'both') & (df_joined['year_truth'].notna() & df_joined['year_result'].isna())].shape[0] + df_joined[(df_joined['_merge'] == 'both') & (df_joined['previous_year_truth'].notna() & df_joined['previous_year_result'].isna())].shape[0],
        "false_negative": df_joined[(df_joined['_merge'] == 'both') & (df_joined['year_truth'].isna() & df_joined['year_result'].notna())].shape[0] + df_joined[(df_joined['_merge'] == 'both') & (df_joined['previous_year_truth'].isna() & df_joined['previous_year_result'].notna())].shape[0],
        "true_negative": df_joined[(df_joined['_merge'] == 'both') & (df_joined['year_truth'].notna() & df_joined['year_result'].notna())].shape[0] + df_joined[(df_joined['_merge'] == 'both') & (df_joined['previous_year_truth'].notna() & df_joined['previous_year_result'].notna())].shape[0]
    }

    df_year_non_na = df_joined[(df_joined['_merge'] == 'both') & (df_joined['year_truth'].notna() & df_joined['year_result'].notna())]
    df_year_non_na = df_year_non_na.drop(
        columns=[col for col in df_year_non_na.columns if 'previous_year' in col],
        errors='ignore'
    )
    try:
        df_year_non_na['relative_numeric_difference'] = (
            (df_year_non_na['year_result'] - df_year_non_na['year_truth']).abs() /
            df_year_non_na['year_truth'].abs().replace(0, 1)
        )
    except OverflowError:
        print(f"OverflowError calculating relative numeric difference")
        
    df_year_non_na['levenstein_distance'] = df_year_non_na.apply(
        lambda row: nltk.edit_distance(
        str(row['year_truth']), str(row['year_result'])
        ),
        axis=1
    )

    df_previous_year_non_na = df_joined[(df_joined['_merge'] == 'both') & (df_joined['previous_year_truth'].notna() & df_joined['previous_year_result'].notna())]
    df_previous_year_non_na = df_previous_year_non_na.drop(
        columns=[col for col in df_previous_year_non_na.columns if col.startswith('year')],
        errors='ignore'
    )
    df_previous_year_non_na['relative_numeric_difference'] = (
        (df_previous_year_non_na['previous_year_result'] - df_previous_year_non_na['previous_year_truth']).abs() /
        df_previous_year_non_na['previous_year_truth'].abs().replace(0, 1)
    )
    df_previous_year_non_na['levenstein_distance'] = df_previous_year_non_na.apply(
        lambda row: nltk.edit_distance(
        str(row['previous_year_truth']), str(row['previous_year_result'])
        ),
        axis=1
    )

    # Attach both columns for detailed analysis
    evaluation['relative_numeric_difference'] = {
        "mean": (df_year_non_na['relative_numeric_difference'].mean() + df_previous_year_non_na['relative_numeric_difference'].mean())/2,
        "median": (pd.concat([df_year_non_na['relative_numeric_difference'], df_previous_year_non_na['relative_numeric_difference']]).median())
    }
    evaluation['levenstein_distance'] = {
        "mean": (df_year_non_na['levenstein_distance'].mean() + df_previous_year_non_na['levenstein_distance'].mean())/2,
        "median": (pd.concat([df_year_non_na['levenstein_distance'], df_previous_year_non_na['levenstein_distance']]).median())
    }
    evaluation['correct_numeric'] = df_year_non_na[df_year_non_na['year_truth'] == df_year_non_na['year_result']].shape[0] + df_previous_year_non_na[df_previous_year_non_na['previous_year_truth'] == df_previous_year_non_na['previous_year_result']].shape[0]
    evaluation['incorrect_numeric'] = df_year_non_na[df_year_non_na['year_truth'] != df_year_non_na['year_result']].shape[0] + df_previous_year_non_na[df_previous_year_non_na['previous_year_truth'] != df_previous_year_non_na['previous_year_result']].shape[0]
    evaluation['total_entries'] = df_joined.shape[0]*2
    evaluation['filepath'] = entry_filepath

    return evaluation

In [30]:
evaluation = evaluate_single_result(df_result, entry)

In [32]:
df_joined = evaluation['df_joined']
df_joined['match'] = df_joined.apply(lambda row: row['year_truth'] == row['year_result'], axis=1)
df_joined

,E1,E2,E3,year_truth,previous_year_truth,year_result,previous_year_result,_merge,match
0,Anlagevermoegen,Immaterielle Vermoegensgegenstaende,Selbst geschaffene gewerbliche Schutzrechte un...,9586.21,5874.80,9586.21,5874.80,both,True
1,Anlagevermoegen,Immaterielle Vermoegensgegenstaende,Geschaefts- oder Firmenwert,7686.89,841.58,7686.89,841.58,both,True
2,Anlagevermoegen,Immaterielle Vermoegensgegenstaende,geleistete Anzahlungen,NaN,NaN,NaN,NaN,both,False
3,Anlagevermoegen,Immaterielle Vermoegensgegenstaende,"entgeltlich erworbene Konzessionen, gewerblich...",3091.58,5868.83,3091.58,5868.83,both,True
4,Anlagevermoegen,Sachanlagen,"Grundstuecke, grundstuecksgleiche Rechte und B...",2900.39,8697.91,2900.39,8697.91,both,True
5,Anlagevermoegen,Sachanlagen,Technische Anlagen und Maschinen,1349.51,2423.71,1349.51,2423.71,both,True
6,Anlagevermoegen,Sachanlagen,"Andere Anlagen, Betriebs- und Geschaeftsaussta...",NaN,NaN,NaN,NaN,both,False
7,Anlagevermoegen,Sachanlagen,geleistete Anzahlungen und Anlagen im Bau,1556.63,5688.00,1556.63,5688.00,both,True
8,Anlagevermoegen,Finanzanlagen,Sonstige Finanzanlagen,69.79,5202.43,69.79,5202.43,both,True
9,Anlagevermoegen,Finanzanlagen,Anteile an verbundenen Unternehmen,NaN,NaN,NaN,NaN,both,False


In [35]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

evaluations = []

n_files = len(pdf_texts)
for idx, entry in enumerate(pdf_texts):
    print(f"Processing file {idx+1}/{n_files}: {entry['filepath']}", end='\r')
    extracted_rows = extract_numbers(entry['text'])
    df_result = pd.DataFrame(extracted_rows)
    evaluation = evaluate_single_result(df_result, entry)
    evaluations.append(evaluation)

In [36]:
with open(f"../../benchmark_results/table_extraction/regex/evaluation_synth_tables.json", "w") as json_file:
    json.dump(
        [{key: value.to_json(orient='records') if hasattr(value, 'to_json') else value for key, value in entry.items()} for entry in evaluations],
        json_file, 
        indent=4, 
        default=str
    )

# real tables

In [37]:
with open(f"../../Python/pdf_texts_real_tables.json", "r") as json_file:
    real_table_texts = json.load(json_file)

pdf_texts = [{'filepath': '../../benchmark_truth/real_tables/'+key.replace('/pvc/benchmark_truth/real_tables/manual_download/', '').replace('/', '__').replace('.pdf', '.xlsx'), 'text': value} for key, value in real_table_texts.items()]
pdf_texts

[{'filepath': '../../benchmark_truth/real_tables/Amt für Statistik Berlin-Brandenburg__AP_Geschaeftsbericht_DE_2016_BBB.xlsx',
  'text': 'Amt für Statistik Berlin-Brandenburg Anstalt des öffentlichen Rechts, Potsdam\r\nBilanz zum 31. Dezember 2016\r\nA K T I V S E I T E 31.12.2016 Vorjahr \r\nEUR EUR EUR \r\nA. ANLAGEVERMÖGEN \r\nI. Immaterielle Vermögensgegenstände \r\nEntgeltlich erworbene Konzessionen, gewerbliche \r\nSchutzrechte und ähnliche Rechte und Werte sowie Lizenzen \r\nan solchen Rechten und Werten 105\xa0541,00 144\xa0713,00 \r\nII. Sachanlagen \r\n1. Grundstücke, grundstücksgleiche Rechte und Bauten \r\neinschließlich der Bauten auf fremden Grundstücken 21\xa0058,00 38\xa0516,00 \r\n2. Andere Anlagen, Betriebs- und Geschäftsausstattung 367\xa0071,00 133\xa0195,00 \r\n388\xa0129,00 171\xa0711,00 \r\nIII. Finanzanlagen \r\n1. Wertpapiere des Anlagevermögens 0,00 2\xa0000\xa0000,00 \r\n2. Sonstige Finanzanlagen 4\xa0400\xa0000,00 1\xa0800\xa0000,00 \r\n4\xa0400\xa0000,00 3\

In [79]:
df_real_table_characteristics = pd.read_csv("../../benchmark_truth/real_tables/table_characteristics.csv")
df_real_table_characteristics['filepath'] = df_real_table_characteristics.apply(lambda x: '../../benchmark_truth/real_tables/' + x['company'] + '__' + x['filename'].replace('.pdf', '.xlsx'), axis=1)
df_real_table_characteristics

,company,filename,T_in_year,T_in_previous_year,sum_same_line,spacer,n_columns,vorjahr,passiva_same_page,header_span,enumeration,vis_seperated_cols,vis_seperated_rows,sum_in_header,year_missing,geschaeftsjahr,filepath
0,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf,0,0,0,,3,1,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Amt für Stat...
1,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2017_BBB.pdf,0,0,0,,3,1,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Amt für Stat...
2,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf,0,0,0,,3,2,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Amt für Stat...
3,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2019_BBB.pdf,0,0,0,,3,0,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Amt für Stat...
4,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2020_BBB.pdf,0,0,0,,3,0,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Amt für Stat...
5,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2021_BBB.pdf,0,0,0,,3,0,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Amt für Stat...
6,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2022_BBB.pdf,0,0,0,,3,0,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Amt für Stat...
7,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2023_BBB.pdf,0,0,0,.,3,0,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Amt für Stat...
8,Berliner Bäder Betriebe,2018_Geschaeftsbericht_BBB.pdf,0,0,0,.,3,0,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Berliner Bäd...
9,Berliner Stadtgüter,BSG-Geschaeftsbericht-2021-druck.pdf,0,0,0,.,3,0,0,1,1,0,0,0,0,0.0,../../benchmark_truth/real_tables/Berliner Sta...


In [86]:
df_real_tables = pd.DataFrame(pdf_texts)
# df_real_tables['filepath'] = df_real_tables['filepath'].apply(lambda x: x.replace('.xlsx', '.pdf') if x.endswith('.xlsx') else x)
df_real_tables = df_real_tables.merge(df_real_table_characteristics, on="filepath", how="left")
df_real_tables

,filepath,text,company,filename,T_in_year,T_in_previous_year,sum_same_line,spacer,n_columns,vorjahr,passiva_same_page,header_span,enumeration,vis_seperated_cols,vis_seperated_rows,sum_in_header,year_missing,geschaeftsjahr
0,../../benchmark_truth/real_tables/Amt für Stat...,Amt für Statistik Berlin-Brandenburg Anstalt d...,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf,0.0,0.0,0.0,,3.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,../../benchmark_truth/real_tables/Amt für Stat...,Amt für Statistik Berlin-Brandenburg Anstalt d...,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2017_BBB.pdf,0.0,0.0,0.0,,3.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,../../benchmark_truth/real_tables/Amt für Stat...,Amt für Statistik Berlin-Brandenburg Anstalt d...,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf,0.0,0.0,0.0,,3.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,../../benchmark_truth/real_tables/Amt für Stat...,30 statistik Berlin Brandenburg\r\nBilanz\r\nG...,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2019_BBB.pdf,0.0,0.0,0.0,,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,../../benchmark_truth/real_tables/Amt für Stat...,6\r\nBilanz\r\nGewinn- und \r\nVerlustrechnung...,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2020_BBB.pdf,0.0,0.0,0.0,,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,../../benchmark_truth/real_tables/Amt für Stat...,6\r\nBilanz\r\nGewinn- und \r\nVerlustrechnung...,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2021_BBB.pdf,0.0,0.0,0.0,,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
6,../../benchmark_truth/real_tables/Amt für Stat...,6\r\nBilanz\r\nGewinn- und \r\nVerlustrechnung...,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2022_BBB.pdf,0.0,0.0,0.0,,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,../../benchmark_truth/real_tables/Amt für Stat...,6\r\nBilanz\r\nGewinn- und \r\nVerlustrechnung...,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2023_BBB.pdf,0.0,0.0,0.0,.,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
8,../../benchmark_truth/real_tables/Berliner Bäd...,68\r\nBILANZ ZUM \r\n31. DEZEMBER 2018\r\nAKTI...,Berliner Bäder Betriebe,2018_Geschaeftsbericht_BBB.pdf,0.0,0.0,0.0,.,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9,../../benchmark_truth/real_tables/Berliner Sta...,Seite 38 \r\nBILANZ\r\nAKTIVA\r\n \r\nA. ANLA...,Berliner Stadtgüter,BSG-Geschaeftsbericht-2021-druck.pdf,0.0,0.0,0.0,.,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [87]:
dict_real_tables = df_real_tables[['filepath', 'text', 'T_in_year', 'T_in_previous_year']].to_dict(orient='records')

In [97]:
evaluations_real_tables = []

for entry in dict_real_tables:
    print(f"Processing file {entry['filepath']}", end='\r')
    extracted_rows = extract_numbers(entry['text'])
    df_result = pd.DataFrame(extracted_rows)
    evaluation = evaluate_single_result(df_result, entry)
    evaluations_real_tables.append(evaluation)


In [98]:
with open(f"../../benchmark_results/table_extraction/regex/evaluation_real_tables.json", "w") as json_file:
    json.dump(
        [{key: value.to_json(orient='records') if hasattr(value, 'to_json') else value for key, value in entry.items()} for entry in evaluations_real_tables],
        json_file, 
        indent=4, 
        default=str
    )
